1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np

In [2]:
classification_data, classification_labels = datasets.make_classification(n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [3]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data # значения признаков
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его   
        prediction = max(classes, key=classes.get)
        return prediction

In [5]:
def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1     # "impurity" - "нечистота", степень неопределенности
    for label in classes:
        p = classes[label] / len(labels) # долю объектов класса в листе
        impurity -= p ** 2 # Критерий Джини
        
    return impurity

In [6]:
def quality(left_labels, right_labels, current_gini):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0]) # для правого (1-p)
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels) # Функционал качества

In [7]:
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

В качестве критерия останова здесь я реализую останов в случае, когда все объекты в листе относятся к одному классу.

In [8]:
def find_best_split(data, labels):

    current_gini = gini(labels) 

    best_quality = 0
    best_t = None 
    best_index = None 
    
    n_features = data.shape[1] 
    
    for index in range(n_features): 
        t_values = [row[index] for row in data] 
        
        for t in t_values: 
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t) 
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [9]:
def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels) 

    if len(set(labels)) == 1: # если все значения относятся к одному классу, возвращаем лист
        
        return Leaf(data, labels) 
    
   
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf): # проверка текущий узел это лист?
        answer = node.prediction # считаем прогноз для листа
        return answer

    if obj[node.index] <= node.t: # если значение признака меньше порога t
        return classify_object(obj, node.true_branch) # рекурсия: отправляем объект в true-ветку
    else:
        return classify_object(obj, node.false_branch) # рекурсия: отправляем объект в false-ветку

In [11]:
def predict(data, tree):
    
    classes = []
    
    for obj in data:
        prediction = classify_object(obj, tree) # определяем ветки для объектов
        classes.append(prediction)
    return classes

In [12]:
from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [13]:
my_tree = build_tree(train_data, train_labels)
print(my_tree)

In [25]:
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 0
Порог 0.16261402870113306
--> True:
  Индекс 1
  Порог -1.5208896621663803
  --> True:
    Индекс 0
    Порог -0.9478301462477035
    --> True:
      Прогноз: 0
    --> False:
      Индекс 0
      Порог -0.4942656820712277
      --> True:
        Индекс 0
        Порог -0.8411814419026544
        --> True:
          Прогноз: 1
        --> False:
          Прогноз: 0
      --> False:
        Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Прогноз: 1


2*.  Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.
В материалах также есть пример скрипта для расчета дерева - Lesson_4_script, им можно воспользоваться аналогично тому, как это было на предыдущих уроках

In [38]:
regression_data, regression_values = datasets.make_regression(n_features = 2, n_informative = 2, random_state=54)

In [41]:
class Leaf_reg:
    
    def __init__(self, data, values):
        self.data = data # значения признаков
        self.values = values  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        #  вернем в качестве предсказания среднее по выборке   
        prediction = np.mean(self.values)
        return prediction

In [42]:
def quality_reg(left_values, right_values, variance):
    
    # доля выборки, ушедшей в левое поддерево
    p = float(left_values.shape[0]) / (left_values.shape[0] + right_values.shape[0]) # для правого (1-p)
    
    return variance - p*np.var(left_values) - (1-p)*np.var(right_values)

In [43]:
def split_reg(data, values, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_values = values[left]
    false_values = values[right]
        
    return true_data, false_data, true_values, false_values

In [45]:
def find_best_split_reg(data, values):
    
    variance = np.var(values) 

    best_quality = 0
    best_t = None 
    best_index = None 
    
    n_features = data.shape[1] 
    
    for index in range(n_features): 
        t_values = [row[index] for row in data] 
        
        for t in t_values: 
            true_data, false_data, true_values, false_values = split_reg(data, values, index, t) 
            
            current_quality = quality_reg(true_values, false_values, variance)
            
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

В качестве критерия останова использовала ограничение максимальной глубины дерева. 

In [75]:
def build_tree_reg(data, values, depth =0, max_depth = 5):
    
    quality, t, index = find_best_split_reg(data, values) 

    if quality == 0 or depth == max_depth:
        
        return Leaf_reg(data, values) 
    
    true_data, false_data, true_values, false_values = split(data, values, index, t)
    
    depth_new = depth + 1
    true_branch = build_tree_reg(true_data, true_values, depth_new)
    false_branch = build_tree_reg(false_data, false_values, depth_new)

    return Node(index, t, true_branch, false_branch)

In [76]:
def classify_object_reg(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf_reg): # проверка текущий узел это лист?
        answer = node.prediction # считаем прогноз для листа
        return answer

    if obj[node.index] <= node.t: # если значение признака меньше порога t
        return classify_object_reg(obj, node.true_branch) # рекурсия: отправляем объект в true-ветку
    else:
        return classify_object_reg(obj, node.false_branch) # рекурсия: отправляем объект в false-ветку

In [77]:
def predict_reg(data, tree):
    
    values = []
    
    for obj in data:
        prediction = classify_object_reg(obj, tree) # определяем ветки для объектов
        values.append(prediction)
    return values

In [78]:
from sklearn import model_selection

train_data_reg, test_data_reg, train_values, test_values = model_selection.train_test_split(regression_data, 
                                                                                     regression_values, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [79]:
my_tree_reg = build_tree_reg(train_data_reg, train_values)
print(my_tree_reg)

In [80]:
def print_tree_reg(node, spacing=""):

    if isinstance(node, Leaf_reg):
        print(spacing + "Прогноз:", node.prediction)
        return

    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    print (spacing + '--> True:')
    print_tree_reg(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree_reg(node.false_branch, spacing + "  ")
    
print_tree_reg(my_tree_reg)

Индекс 0
Порог -0.1233520210831071
--> True:
  Индекс 0
  Порог -0.9540273840395093
  --> True:
    Индекс 0
    Порог -1.6337823062115282
    --> True:
      Индекс 1
      Порог -1.4709785555691692
      --> True:
        Прогноз: -88.16108547574878
      --> False:
        Индекс 0
        Порог -1.9938757057055059
        --> True:
          Прогноз: -73.32627770685623
        --> False:
          Прогноз: -64.43375164538678
    --> False:
      Индекс 1
      Порог 0.07856382551083278
      --> True:
        Индекс 1
        Порог -0.9450240252146124
        --> True:
          Прогноз: -58.89641831306484
        --> False:
          Прогноз: -49.73920261023643
      --> False:
        Индекс 0
        Порог -1.5500218326779363
        --> True:
          Прогноз: -29.287220596528666
        --> False:
          Прогноз: -19.839425469609186
  --> False:
    Индекс 1
    Порог 0.6245624608756769
    --> True:
      Индекс 1
      Порог -0.40425156235749193
      --> True:
        И